In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
import random
from tensorflow import keras

In [2]:
import keras
from keras.callbacks import ModelCheckpoint
from keras.layers import TimeDistributed as td
from keras.layers import Input, Conv2D, Flatten, Dense, ZeroPadding2D, Activation, concatenate
from keras.layers import MaxPooling2D, Dropout, BatchNormalization, Reshape, UpSampling2D
from keras.models import Model

In [3]:
# Convoltuion blocks
def conv_block(tensor, nfilters, size=3, padding='same', kernel_initializer = 'he_normal'):
    x = Conv2D(filters=nfilters, kernel_size=(size,size) , padding=padding, kernel_initializer = 'he_normal')(tensor)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)
    return x
  
def deconv_block(tensor, residual, nfilters, size=3, padding='same', kernel_initializer = 'he_normal'):
    y = UpSampling2D(size = (2,2))(tensor)
    y = Conv2D(filters=nfilters, kernel_size=(size,size), activation = 'relu', padding = 'same', kernel_initializer = kernel_initializer)(y)
    y = concatenate([y,residual], axis = 3)
    y = conv_block(y, nfilters)
    
    return y   
     

In [4]:
def get_jnet():
    inputs = Input((224,224,3))
    
    #Contraction path
    conv1= conv_block(inputs, 16)
    pool1 = MaxPooling2D(pool_size=2)(conv1)

    conv2 = conv_block(pool1, 32)
    pool2 = MaxPooling2D(pool_size=2)(conv2)

    conv3 = conv_block(pool2, 64)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)
    
    conv4 = conv_block(pool3, 128)
    pool4 = MaxPooling2D(pool_size=2)(conv4)
   
    conv5 = conv_block(pool4, 256)
   
    #Expansion path
    up6 = deconv_block(conv5, conv4, 128)
    up7 = deconv_block(up6, conv3, 64)
    up8 = deconv_block(up7, conv2, 32)
    up9= deconv_block(up8, conv1, 16)
    
    pool_fin = MaxPooling2D(pool_size=2)(up9)
    conv10 = Conv2D(16, kernel_size=(28, 28), strides=(28, 28))(pool_fin)
    out = BatchNormalization()(conv10)
    out = Reshape((16, 16))(out)
    out = Activation('softmax')(out)

    model = Model(inputs=[inputs], outputs=[out])

    return model

In [5]:
model = get_jnet()
adam = keras.optimizers.Adam(learning_rate=.001)
model.compile(optimizer=adam, loss='sparse_categorical_crossentropy',
              metrics = ['sparse_categorical_accuracy'])

In [6]:
def load_data(base_path, path, pieces=2):
    data = pd.read_csv(base_path + '{}.csv'.format(path))
    path = '/train/TRAIN_'

    x, y = [], []
    for i in range(len(data)):
        im = Image.open(f"{path}{i:05d}"+ '.jpg')
        im = np.array(im).astype('float16')
        im = im / 255 - 0.5

        x.append(im)

        label = data.iloc[i]
        label = [int(i) for i in label]
        y.append(label)

    return (np.array(x), np.expand_dims(np.array(y), axis=-1))

In [7]:
def group_image(images, shuffle_arr, pieces=4):
  img = []

  new_array = [shuffle_arr[i] for i in shuffle_arr]
  print(new_array)
  new_order =  [images[i] for i in new_array]

  for i in range(pieces):
    img.append(np.concatenate(new_order[i*pieces:(i+1)*pieces], axis=1))
    
  img = np.concatenate(img, axis=0)
  return img

In [8]:
base_path = "/"

In [9]:
x_train, y_train = load_data(base_path, 'train', pieces=4)

FileNotFoundError: [Errno 2] No such file or directory: '/train.csv'